# 1. 특정 폴더에 패키지 설치
파이썬 패키지를 활용하기 위해서는 특정 폴더에 해당 패키지 파일을 설치하고,
해당 폴더를 사용하겠다고 선언해줘야한다.

In [1]:
# 특정 폴더에 패키지 파일 설치 예시

# 패키지 파일을 설치할 폴더
# 띄어 쓰기가 있는 경우, cmd 에서는 전체를 쌍따옴표(")로 감싼다.
mypath = "C:/Program Files/Python38/"

# 주피터 노트북에서는 powershell 에서 구동되므로 띄어쓰기 있는 부분만 쌍따옴표(")로 감싼다.
mypath = 'C:/"Program Files"/Python38/'

# ! : Terminal, CMD, Powershell 등에서 명령어를 실행한다.
# pip : 파이썬으로 작성된 패키지 소프트웨어를 설치/관리하는 패키지 관리 시스템. 파이썬 3.4 이후 버전은 pip를 기본적으로 포함함

# pip list는 현재 설치된 패키지를 확인할 수 있음
!pip list

# # pip install 특정 폴더에 특정 패키지를 설치할 수 있음
# # --target=$myapth 부분 없이 pip install schedule 을 하면 기본 폴더에 설치되지만, target으로 특정 폴더에 설치하고 활용하는 것을 권장함.
# !pip install --target=$mypath schedule

# # pip를 업그레이드 할 수 있음
# !pip install --target=$mypath --upgrade pip

Package            Version
------------------ ---------
appdirs            1.4.4
backcall           0.2.0
beautifulsoup4     4.10.0
bs4                0.0.1
certifi            2021.5.30
charset-normalizer 2.0.6
colorama           0.4.4
cssselect          1.1.0
cycler             0.11.0
DateTime           4.3
debugpy            1.4.1
decorator          5.0.9
Deprecated         1.2.12
fake-useragent     0.1.11
feedparser         6.0.8
finance-datareader 0.9.31
html5lib           1.1
idna               3.2
importlib-metadata 4.8.1
ipykernel          6.1.0
ipython            7.26.0
ipython-genutils   0.2.0
jedi               0.18.0
jupyter-client     6.1.12
jupyter-core       4.7.1
kiwisolver         1.3.2
lxml               4.6.3
matplotlib         3.4.3
matplotlib-inline  0.1.2
multitasking       0.0.9
mysql              0.0.3
mysql-connector    2.2.9
mysqlclient        2.0.3
numpy              1.21.2
pandas             1.3.3
pandas-datareader  0.10.0
parse              1.19.0
parso     

In [2]:
import time
import threading
import schedule

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta # 특정 날짜 계산하는 패키지
import pandas as pd
import numpy as np
import yfinance as yf
from yahooquery import Ticker
import yahoo_fin.stock_info as si

# 슬랙 알람 울리기
import requests
from public_data_config import apisdata

def post_message(token, channel, text):
    response = requests.post("https://slack.com/api/chat.postMessage",
        headers={"Authorization": "Bearer "+token},
        data={"channel": channel,"text": text}
    )
    print(response)
 
myToken = apisdata['slack']['token']
channel = apisdata['slack']['channel']
 
# 메세지 보내기 테스트
# post_message(myToken,channel,"test_message")

In [3]:
import pymysql
from db_config import config

conn = pymysql.connect(
    host=config['host'], 
    user=config['user'],
    password=config['password'],
    database=config['database'],
    port=config['port'])
cur = conn.cursor(pymysql.cursors.DictCursor)

sql = """
select * from trading;
"""

cur.execute(sql)

rows = cur.fetchall()
cur.close()
conn.close()

In [4]:
df = pd.DataFrame.from_dict(rows)
df.head()

,id,account_number,symbol,name,event_datetime,timestamp,action,price,quantity,division
0,1,5100-4293,AMZN,아마존닷컴,2021-12-31,None,buy,2553.340,1,None
1,2,5100-4293,GOOG,알파벳C,2021-12-31,None,buy,210.800,1,None
2,3,5100-4293,COIN,코인베이스 글로벌,2021-12-31,None,buy,365.500,2,None
3,4,5100-4293,SPY,SPDR S&P 500,2021-12-31,None,buy,282.405,2,None
4,5,5100-4293,DIA,SPDR DJ INDUSTRIAL AVERAGE ETF,2021-12-31,None,buy,229.220,10,None


In [7]:
df['price'] = np.where(df['symbol']=='GOOG', 2108, df['price'])

In [8]:
df.head()

,id,account_number,symbol,name,event_datetime,timestamp,action,price,quantity,division
0,1,5100-4293,AMZN,아마존닷컴,2021-12-31,None,buy,2553.340,1,None
1,2,5100-4293,GOOG,알파벳C,2021-12-31,None,buy,2108.000,1,None
2,3,5100-4293,COIN,코인베이스 글로벌,2021-12-31,None,buy,365.500,2,None
3,4,5100-4293,SPY,SPDR S&P 500,2021-12-31,None,buy,282.405,2,None
4,5,5100-4293,DIA,SPDR DJ INDUSTRIAL AVERAGE ETF,2021-12-31,None,buy,229.220,10,None


In [9]:
df['total_price'] = df['price'] * df['quantity']
g_df = df[df['action'] == 'buy'].groupby(['symbol'])[['quantity', 'total_price']].sum().reset_index()
g_df['mean_buy_price'] = g_df['total_price'] / g_df['quantity']

In [10]:
info_columns = ['symbol', 'shortName', 'sharesOutstanding', 'floatShares', 'currentPrice', 'averageVolume', 'averageDailyVolume10Day']
idata = []
for t in g_df.symbol.unique().tolist():
    print(t, 'start')
    start_time = datetime.now()
    t_dict = dict()
    info = yf.Ticker(t).info
    for c in info_columns:
        if c not in t_dict:
            t_dict[c] = None
        if c in t_dict:
            try:
                t_dict[c] = info[c]
            except:
                try:
                    sd = Ticker(t).summary_detail
                    if c in sd[t].keys():
                        t_dict[c] = t[c]
                    else:
                        if c == 'currentPrice':
                            t_dict[c] = t['previousClose']
                        elif c == 'sharesOutstanding':
                            t_dict[c] = t['']
                except:
                    pass
    t_dict['event_datetime'] = datetime.now()
    idata.append(t_dict)
    print('end', (datetime.now() - start_time).seconds, 'seconds')
    
volume_df = pd.DataFrame(idata)
volume_df.head()

AAPL start
end 4 seconds
ABB start
end 3 seconds
AMZN start


KeyboardInterrupt: 

In [11]:
volume_df.to_pickle('C:/Users/is/Downloads/volume_df_220106.pkl')

NameError: name 'volume_df' is not defined

In [12]:
v_df = pd.read_pickle('C:/Users/is/Downloads/volume_df_220106.pkl')
v_df

,symbol,shortName,sharesOutstanding,floatShares,currentPrice,averageVolume,averageDailyVolume10Day,event_datetime
0,AAPL,Apple Inc.,1.668760e+10,1.638966e+10,179.7000,89365539.0,79559520,2022-01-06 01:04:48.826001
1,ABB,ABB Ltd,1.993000e+09,1.618959e+09,39.0500,1424100.0,1108160,2022-01-06 01:04:54.931553
2,AMZN,"Amazon.com, Inc.",5.043240e+08,4.543480e+08,3350.4400,3172961.0,2581830,2022-01-06 01:05:02.270781
3,BA,Boeing Company (The),5.848100e+08,5.374451e+08,213.6300,9610442.0,8595940,2022-01-06 01:05:10.144122
4,BAC,Bank of America Corporation,8.569320e+09,8.156749e+09,47.9900,45448304.0,39051950,2022-01-06 01:05:17.550359
5,BBD,Banco Bradesco Sa,1.069100e+10,6.931072e+09,3.4500,37208112.0,28502450,2022-01-06 01:05:25.001109
6,BNKU,MicroSectors U.S. Big Banks Ind,NaN,NaN,NaN,191003.0,218910,2022-01-06 01:05:34.007176
7,COIN,"Coinbase Global, Inc.",1.417900e+08,1.348616e+08,250.1500,5319519.0,3519020,2022-01-06 01:05:39.314084
8,CZR,"Caesars Entertainment, Inc.",2.086990e+08,2.039341e+08,89.9500,2176195.0,1691300,2022-01-06 01:05:47.327659
9,DAL,"Delta Air Lines, Inc.",6.396470e+08,6.351438e+08,40.5900,13524268.0,10539810,2022-01-06 01:05:54.808054


In [13]:
a_df = pd.merge(g_df, v_df, how='left', on='symbol')
a_df.head()

,symbol,quantity,total_price,mean_buy_price,shortName,sharesOutstanding,floatShares,currentPrice,averageVolume,averageDailyVolume10Day,event_datetime
0,AAPL,20,2627.320,131.366000,Apple Inc.,1.668760e+10,1.638966e+10,179.70,89365539.0,79559520,2022-01-06 01:04:48.826001
1,ABB,86,2035.802,23.672116,ABB Ltd,1.993000e+09,1.618959e+09,39.05,1424100.0,1108160,2022-01-06 01:04:54.931553
2,AMZN,1,2553.340,2553.340000,"Amazon.com, Inc.",5.043240e+08,4.543480e+08,3350.44,3172961.0,2581830,2022-01-06 01:05:02.270781
3,BA,30,4345.770,144.859000,Boeing Company (The),5.848100e+08,5.374451e+08,213.63,9610442.0,8595940,2022-01-06 01:05:10.144122
4,BAC,20,940.082,47.004100,Bank of America Corporation,8.569320e+09,8.156749e+09,47.99,45448304.0,39051950,2022-01-06 01:05:17.550359


In [14]:
a_df['sharesReleaseRatio'] = a_df['floatShares'] / a_df['sharesOutstanding'] * 100
a_df['holding_ratio'] = a_df['quantity'] / a_df['sharesOutstanding'] * 100
a_df['rate_of_return'] = (a_df['currentPrice'] - a_df['mean_buy_price']) / a_df['mean_buy_price'] *100

In [17]:
a_df.sort_values(['holding_ratio'], ascending=[False])

,symbol,quantity,total_price,mean_buy_price,shortName,sharesOutstanding,floatShares,currentPrice,averageVolume,averageDailyVolume10Day,event_datetime,sharesReleaseRatio,holding_ratio,rate_of_return
36,VLDR,117,1060.9840,9.068239,"Velodyne Lidar, Inc.",1.896850e+08,1.003491e+08,4.9000,3675025.0,2743640,2022-01-06 01:09:27.183580,52.903025,6.168121e-05,-45.965255
20,LOAN,4,25.7200,6.430000,"Manhattan Bridge Capital, Inc",9.619940e+06,8.720410e+06,5.6300,49450.0,58050,2022-01-06 01:07:21.995359,90.649318,4.158030e-05,-12.441680
28,QS,79,3702.8100,46.871013,QuantumScape Corporation,2.596390e+08,2.470385e+08,22.9700,11082268.0,7130980,2022-01-06 01:08:22.904013,95.146926,3.042686e-05,-50.993165
27,PBA,101,1860.2399,18.418217,Pembina Pipeline Corp.,5.499580e+08,5.493804e+08,30.4000,927542.0,730930,2022-01-06 01:08:14.937815,99.894976,1.836504e-05,65.053980
34,UAL,35,1723.5100,49.243143,"United Airlines Holdings, Inc.",3.235780e+08,3.229258e+08,46.2500,12600468.0,11413900,2022-01-06 01:09:11.542392,99.798435,1.081656e-05,-6.078294
23,NKLA,24,862.6416,35.943400,Nikola Corporation,4.068850e+08,2.337424e+08,10.8650,12625141.0,20659730,2022-01-06 01:07:41.605345,57.446806,5.898473e-06,-69.771919
9,DAL,37,1611.3096,43.548908,"Delta Air Lines, Inc.",6.396470e+08,6.351438e+08,40.5900,13524268.0,10539810,2022-01-06 01:05:54.808054,99.295991,5.784441e-06,-6.794448
3,BA,30,4345.7700,144.859000,Boeing Company (The),5.848100e+08,5.374451e+08,213.6300,9610442.0,8595940,2022-01-06 01:05:10.144122,91.900802,5.129871e-06,47.474441
13,EHI,1,10.1727,10.172700,Western Asset Global High Incom,2.269730e+07,NaN,9.8400,46569.0,57760,2022-01-06 01:06:28.083722,NaN,4.405810e-06,-3.270518
1,ABB,86,2035.8020,23.672116,ABB Ltd,1.993000e+09,1.618959e+09,39.0500,1424100.0,1108160,2022-01-06 01:04:54.931553,81.232272,4.315103e-06,64.962015


In [18]:
from yahooquery import Ticker
t = Ticker('BNKU')

In [25]:
t.summary_detail

{'BNKU': {'maxAge': 1,
  'priceHint': 2,
  'previousClose': 66.01,
  'open': 66.85,
  'dayLow': 65.8108,
  'dayHigh': 67.66,
  'regularMarketPreviousClose': 66.01,
  'regularMarketOpen': 66.85,
  'regularMarketDayLow': 65.8108,
  'regularMarketDayHigh': 67.66,
  'volume': 199766,
  'regularMarketVolume': 199766,
  'averageVolume': 191003,
  'averageVolume10days': 218910,
  'averageDailyVolume10Day': 218910,
  'bid': 66.09,
  'ask': 66.13,
  'bidSize': 2900,
  'askSize': 1000,
  'yield': 0.0,
  'totalAssets': 117797040,
  'fiftyTwoWeekLow': 23.02,
  'fiftyTwoWeekHigh': 71.35,
  'fiftyDayAverage': 60.4696,
  'twoHundredDayAverage': 55.1461,
  'navPrice': 62.25,
  'currency': 'USD',
  'fromCurrency': None,
  'toCurrency': None,
  'lastMarket': None,
  'algorithm': None,
  'tradeable': False}}

In [ ]:
t.summary_detail[info_columns[3]]

In [ ]:
for row in rows:
    print(row)
    break

In [ ]:
import FinanceDataReader as fdr
df = fdr.EtfListing('KR')
df.head()

In [ ]:
df = yf.download(tickers='DVN', period='1d', interval='1m')
df.tail()

In [ ]:
market_dict = {
    'dow' : None,
    'sp500' : None,
    'nasdaq' : None
}

# 시장별 티커 리스트 가져오기
for market in market_dict:
    tickers = None

    # dow 지수 ticker list
    if market == 'dow':
        tickers = si.tickers_dow()
        market_dict[market] = tickers
        print('%s : %s'%(market, len(tickers)))

    # sp500 지수 ticker list
    elif market == 'sp500':
        tickers = si.tickers_sp500()
        market_dict[market] = tickers
        print('%s : %s'%(market, len(tickers)))

    # nasdaq 지수 ticker list
    elif market == 'nasdaq':
        tickers = si.tickers_nasdaq()
        market_dict[market] = tickers
        print('%s : %s'%(market, len(tickers)))


In [ ]:
yf.Tickers(market_dict['dow']).history(period='1y')['Close']

In [ ]:
# 기초 성과 지표 만들기

def create_basic_performance_indicators(market, market_ticker_list):

    # NOTE : 금융 공학에서의 1년 동안의 일수 = 252
    financial_engineering_days = 252

    i_dict = {}
    gagr = None
    mdd = None
    vol = None
    sharpe = None

    try:
        # Get tickers close price
        df = yf.Tickers(market_ticker_list).history(period='1y')['Close']
            
    except Exception as e:
        error_message = '%s error : %s'%(market, e)
        post_message(myToken, channel, error_message)

    for ticker in df.columns.tolist():
        if ticker not in i_dict:
            i_dict[ticker] = {}

        if ticker in i_dict:
            recent_date = df.index[-1]
            if recent_date not in i_dict[ticker]:
                i_dict[ticker][recent_date] = {
                    'gagr' : None,
                    'mdd' : None,
                    'vol' : None,
                    'sharpe' : None
                }

            if recent_date in i_dict[ticker]:
                ticker_df = df[ticker].reset_index()
                # 1. GAGR(Compound Annual Growth Rate, 연평균 복리 수익률)
                # 1.1. 수정된 종가 데이터로 일별 수익률 생성하기
                ticker_df['daily_rtn'] = ticker_df[ticker].pct_change()  # 퍼센트 변화율

                # 1.2. 일별 수익률로 전략 수익률 계산하기
                ticker_df['st_rtn'] = (1 + ticker_df['daily_rtn']).cumprod()  # 누적 곱 계산 함수 return cumulative product over a DataFrame or Series axis.

                # 1.3. GAGR(Compound Annual Growth Rate, 연평균 복리 수익률) 계산하기
                # NOTE : 단위 %
                gagr = (ticker_df[ticker_df['Date']==recent_date]['st_rtn'] ** (financial_engineering_days / len(ticker_df.index)) - 1) * 100

                # 2. MDD(Max Draw Down, 최대 낙폭)
                # 2.1. 관측 기간 최고점 가격 계산
                historical_max = ticker_df[ticker].cummax()  # 관측 기간 동안 누적 최댓값

                # 2.2 일별 감소비율 계산
                daily_drawdown = ticker_df[ticker] / historical_max - 1.0  # 현재 수정 종가에서 누적 최댓값 대비 낙폭률 계산

                # 2.3 일별 최대 하락률 계산
                historical_dd = daily_drawdown.cummin()  

                # 2.4 MDD 계산하기
                # NOTE : 단위 %
                mdd = - 1 * historical_dd.min() * 100

                # 3. Vol(Volatility, 변동성)
                # NOTE 단위 % 
                vol = np.std(ticker_df['daily_rtn']) * np.sqrt(financial_engineering_days) * 100

                # 4. sharpe(ex-post Sharpe ratio, 사후적 사프 비율)
                # NOTE : 샤프 지수는 위험성 대비 수익성 지표. 사후적 샤프 비율은 자산의 실현 수익률을 사용함
                # 실현 수익률의 산술 평균 / 실현 수익률의 변동성으로 계산함
                # 1 보다 크면 좋은 것으로 평가
                sharpe = np.mean(ticker_df['daily_rtn']) / np.std(ticker_df['daily_rtn']) * np.sqrt(financial_engineering_days)

                # dictionary 에 계산 값 입력하기
                i_dict[ticker][recent_date]['gagr'] = gagr
                i_dict[ticker][recent_date]['mdd'] = mdd
                i_dict[ticker][recent_date]['vol'] = vol
                i_dict[ticker][recent_date]['sharpe'] = sharpe

    complete_message = '%s Complete basic_performance_indicators : %s'%(market, len(list(i_dict.keys())))
    post_message(myToken,channel, complete_message)
    return i_dict

In [ ]:
market = 'dow'
dow_basic_index = create_basic_performance_indicators(market, market_dict[market])

In [ ]:
market = 'sp500'
sp500_basic_index = create_basic_performance_indicators(market, market_dict[market])
sp500_basic_index

In [ ]:
def create_basic_index_dataframe(basic_index_dict):
    ilist = []
    dict = {}
    for ticker in basic_index_dict:
        try:
            recent_date = list(basic_index_dict[ticker].keys())[0]
        except Exception as e:
            print(ticker, e)
            pass
        gagr = basic_index_dict[ticker][recent_date]['gagr']
        mdd = basic_index_dict[ticker][recent_date]['mdd']
        vol = basic_index_dict[ticker][recent_date]['vol']
        sharpe = basic_index_dict[ticker][recent_date]['sharpe']
        dict = {
            'ticker' : ticker,
            'recent_date' : recent_date,
            'gagr' : gagr,
            'mdd' : mdd,
            'vol' : vol,
            'sharpe' : sharpe
        }
        
        ilist.append(dict)
    df = pd.DataFrame(ilist)
    return df

In [ ]:
sp500_df = create_basic_index_dataframe(sp500_basic_index)
sp500_df.tail()

In [ ]:
candidate_list = sp500_df[(sp500_df['sharpe'] > 1) & (sp500_df['vol'] > 40)].sort_values(by='sharpe', ascending=False)['ticker'].tolist()
print(len(candidate_list))
candidate_list

In [ ]:
def get_live_price(candidate_list):
    try:
        df = yf.download(tickers=candidate_list, period='1', interval='1m')
    except Exception as e:
        print(e)
        pass
    close_column_name = 'Adj Close'
    adjclose_df = df[close_column_name]
    for ticker in adjclose_df.columns.tolist():
        ticker_df = adjclose_df[ticker].reset_index().rename(columns={ticker:close_column_name})
        ticker_df['recent_rtn'] = ticker_df[close_column_name].pct_change()
        ticker_df['st_rtn'] = ((1 + ticker_df['recent_rtn']).cumprod() - 1) * 100   # 누적 곱 계산 함수 return cumulative product over a DataFrame or Series axis.
        if ticker_df[ticker_df[close_column_name].isna() == False].iloc[-1]['st_rtn'] < -1:
            buy_message = 'Buy Now!! Ticker : %s, Recent Price : %s'%(ticker, ticker_df[ticker_df[close_column_name].isna() == False].iloc[-1][close_column_name])
            print(buy_message)
            post_message(myToken, channel, buy_message)
            pass

In [ ]:
get_live_price(candidate_list)

In [ ]:
alb = yf.Ticker('ALB')
alb

In [ ]:
alb.info

In [ ]:
alb.actions

In [ ]:
alb.dividends

In [ ]:
alb.financials

In [ ]:
alb.major_holders

In [ ]:
# 일정 시간마다 특정 함수 수행 스케줄링
schedule.every(1).minutes.do(get_live_price, candidate_list)
# ticker = get_ticker()
# schedule.every(1).minutes.do(get_market_status, ticker)
# market_status, earnings_date = get_market_status(ticker)
# text = 'make_time : %s, market_status : %s, ticker : %s, next_earning_date : %s'%(datetime.now(), market_status, ticker, earnings_date)
# schedule.every(1).minutes.do(post_message,text)

# schedule.every().day.at("11:08").do(post_message,text)

while True:
    print('try : ', datetime.now())
    schedule.run_pending()
    time.sleep(60)
    

In [ ]:
#  특정 티커별 데이터 가져오기
def get_certain_ticker_data(certain_ticker):
    df = si.get_data(certain_ticker)
    
    # 1. 바이앤 홀드 전략
    # 수정된 종가 데이터 가져오기
    bh_df = df.loc[:, ['adjclose']].copy()

    # 수정된 종가 데이터로 일별 수익률 생성하기
    bh_df['daily_rtn'] = bh_df['adjclose'].pct_change()  # 퍼센트 변화율
    
    # 일별 수익률로 전략 수익률 계산하기
    bh_df['st_rtn'] = (1 + bh_df['daily_rtn']).cumprod()
    
    return bh_df

df = get_certain_ticker_data('AAPL')
df.head()

In [ ]:
# buy and hold strategy's returns arithmetic mean
base_date = '2021-11-01'
tmp_df = df.loc[base_date:,['st_rtn']] / df.loc[base_date, ['st_rtn']]
last_date = tmp_df.index[-1]
bh_ar_mean = round((tmp_df.loc[last_date, 'st_rtn'] - 1)*100, 3)

In [ ]:
# buy and hold strategy's 
bh_ge_mean = df.loc[base_date, 'st_rtn']

In [ ]:
round((tmp_df.loc[last_date, 'st_rtn'] - 1)*100, 3)

In [ ]:
tmp_df.loc[last_date, 'st_rtn']

In [ ]:
df.loc[base_date:,['st_rtn']] / df.loc[base_date, ['st_rtn']]

In [ ]:
df.loc[base_date, ['st_rtn']]

In [ ]:
price_df['daily_rtn'] = price_df['adjclose'].pct_change()
price_df['st_rtn'] = (1 + price_df['daily_rtn']).cumprod()
price_df.head(10)

In [ ]:
# 1. 평균 회귀 전략
n = 20
sigma = 2
def bollinger_band(price_df, n, sigma):
    bb = price_df.copy()
    bb['center'] = bb['adjclose'].rolling(n).mean()  # 중앙 이동 평균선
    bb['ub'] = bb['center'] + sigma * bb['adjclose'].rolling(n).std()  # 상단 밴드
    bb['lb'] = bb['center'] - sigma * bb['adjclose'].rolling(n).std()  # 하단 밴드
    return bb

bollinger = bollinger_band(price_df, n, sigma)
bollinger

In [ ]:
base_date = '2021-09-01'
sample = bollinger.loc[base_date:]
sample.head()

In [ ]:
book = sample[['adjclose']].copy()
book['trade'] = ''
book.head()

In [ ]:
def create_trade_book(sample):
    book = sample[['adjclose']].copy()
    book['trade'] = ''
    return book


In [ ]:
def tradings(sample, book):
    for i in sample.index:
        if sample.loc[i, 'adjclose'] > sample.loc[i, 'ub']:  # 상단 밴드 이탈 시 동작 안함
            book.loc[i, 'trade'] = ''
        elif sample.loc[i, 'lb'] > sample.loc[i, 'adjclose']:  # 하방 밴드 이탈 시 매수
            if book.shift(1).loc[i, 'trade'] == 'buy':  # 이미 매수 상태라면
                book.loc[i, 'trade'] = 'buy'  # 매수 상태 유지
            else:
                book.loc[i, 'trade'] = 'buy'
        elif sample.loc[i, 'ub'] >=  sample.loc[i, 'adjclose'] and sample.loc[i, 'adjclose'] >= sample.loc[i, 'lb']:  # 볼린저 밴드 안에 있을 시
            if book.shift(1).loc[i, 'trade'] == 'buy':
                book.loc[i, 'trade'] = 'buy'  # 매수 상태 유지
            else:
                book.loc[i, 'trade'] == ''  # 동작 안함
    return book


In [ ]:
book = tradings(sample, book)
book.tail(10)

In [ ]:
def returns(book):
    # 손익 계산
    rtn = 1.0
    book['return'] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index:
        # long 진입
        if book.loc[i, 'trade'] == 'buy' and book.shift(1).loc[i, 'trade'] == '':
            buy = book.loc[i, 'adjclose']
            print('진입일 : ', i, 'long 진입 가격 : ', buy)
        # long 청산
        elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy':
            sell = book.loc[i, 'adjclose']
            rtn = (sell - buy) / buy + 1 #  손익 계산
            book.loc[i, 'return'] = rtn
            print('청산일 : ', i, 'long 진입 가격 : ', buy, 'long 청산 가격 : ', sell, '| return : ', round(rtn, 4))
        if book.loc[i, 'trade'] == '':  # 제로 포지션
            buy = 0.0
            sell = 0.0
    acc_rtn = 1.0
    for i in book.index:
        rtn = book.loc[i, 'return']
        acc_rtn =  acc_rtn * rtn  # 누적 수익률 계산
        book.loc[i, 'acc_return']  = acc_rtn
    print('Accumulated return : ', round(acc_rtn, 4))
    return (round(acc_rtn, 4))

In [ ]:
returns(book)

In [ ]:
temp_df = price_df.loc[base_date:, ['st_rtn']] / price_df.loc[base_date,['st_rtn']]
last_date = temp_df.index[-1]
temp_df.loc[last_date, 'st_rtn']

In [ ]:
last_date

In [ ]:
base_date

In [ ]:
range = (last_date - datetime.strptime(base_date, '%Y-%m-%d')).days
CAGR = price_df.loc[base_date, 'st_rtn'] ** (range/len(price_df.loc[base_date:].index)) - 1
CAGR

In [ ]:
print(base_date, last_date)

In [ ]:
import matplotlib.pylab as plt
book['acc_return'].plot()

In [ ]:
# 2. 듀얼 모멘텀 전략
t_df = pd.DataFrame()

In [ ]:
si.get_stats(dow_list[0])

In [ ]:
t_dict = si.get_analysts_info('AMZN')
analysts_list = list(t_dict.keys())
analysts_list

In [ ]:
i = 2
print(analysts_list[i])
t_dict[analysts_list[i]]

In [ ]:
t_dict['Earnings Estimate']

In [ ]:
# slack message 보내기
import requests
from public_data_config import apisdata

def post_message(text):
    token=apisdata['slack']['token']
    channel = apisdata['slack']['channel']
    response = requests.post("https://slack.com/api/chat.postMessage",
        headers={"Authorization": "Bearer "+token},
        data={"channel": channel,"text": text}
    )
    print(response, datetime.now())

def get_ticker():
    ticker = 'MSFT'
    return ticker

def get_market_status(ticker):
    market_status = si.get_market_status()
    earnings_date = si.get_next_earnings_date(ticker)
    return market_status, earnings_date

In [ ]:
# 일정 시간마다 특정 함수 수행 스케줄링
schedule.every(1).minutes.do(get_ticker)
ticker = get_ticker()
schedule.every(1).minutes.do(get_market_status, ticker)
market_status, earnings_date = get_market_status(ticker)
text = 'make_time : %s, market_status : %s, ticker : %s, next_earning_date : %s'%(datetime.now(), market_status, ticker, earnings_date)
schedule.every(1).minutes.do(post_message,text)

# schedule.every().day.at("11:08").do(post_message,text)

while True:
    print('try : ', datetime.now())
    schedule.run_pending()
    time.sleep(10)
    

In [ ]:
dow_list = si.tickers_dow() ## 30개
nasdaq_list = si.tickers_nasdaq() ## 5066개
sp500_list = si.tickers_sp500()  ## 505개

"""
dow_list 와 nasdaq_list 의 중복 ticker는 APPL, AMGN, CSCO, HON, INTC, MSFT, WBA
dow_list 에 들어있는 모든 30개사는 sp500에 포함되어 있음


"""


dow_list

In [ ]:
ticker_list = dow_list + nasdaq_list + sp500_list
ticker_list - list(set(ticker_list))

# 분석 시작일

date_to = datetime.now().date() - timedelta(days=3)

# 과거 데이터 확인 기간
past_data_years = 1
delta = relativedelta(years=past_data_years)

# 데이터 시작일
date_from = date_to - delta
str_date_from = date_from.strftime('%Y-%m-%d')
str_date_to = date_to.strftime('%Y-%m-%d')
print('데이터 시작일 : ', str_date_from, str_date_to)

# 시장 종류
# 'S&P500', 'NASDAQ', 'NYSE', 'AMEX', 'SSE', 'SZSE', 'HKEX', 'TSE', 'HOSE', 'KRX'
market = 'NASDAQ'

In [ ]:
# 시장별 ticker 가져오기
market_list = ['S&P500', 'NASDAQ', 'NYSE', 'KRX']

ticker_dict = dict()
market_df = pd.DataFrame()

for i in range(len(market_list)):
    
    market = market_list[i]
    df = pd.DataFrame()
    if i == 0:
        df = fdr.StockListing(market)
        df['Symbol'] = df['Symbol'].str.replace('/.', '')
        df['Market'] = market
        market_df = df
        print(market_list[i])
    else:
        df = fdr.StockListing(market)
        df['Symbol'] = df['Symbol'].str.replace('/.', '')
        df['Market'] = market
        market_df = market_df.append(df)
        print(market_list[i])
market_df.to_pickle()

In [ ]:
market_df.groupby('Market')['Symbol'].nunique()

In [ ]:
def make_inverstment_performance_indicator(df, target_price, date):
    daily_rtn = 'daily_rnt' # 일별 수익률
    st_rtn = 'st_rtn'  # 전략 수익률 수익률 누적의 곱

    df[daily_rtn] = df[target_price].pct_change()  # 일별 수익률
    df[st_rtn] = (1 + df[daily_rtn]).cumprod()  # 전략 수익률 = 수익률의 누적 곱
    
    # CAGR = Compound Annual Growth Rate 연평균 복리 수익률, 수익률이 높을 수록 좋다.
    CAGR = df.loc[date, st_rtn] ** (252./len(df.index)) -1  # 특정 시점까지의  최종 누적 수익률의 누적 연도 제곱근, 금융공학에서 1년은 252 영업일

    # MDD = Maximum Draw Down 최대 낙폭, MDD 값이 크면 변동성이 심한 주식
    historical_max = df[target_price].cummax()
    daily_drawdown = df[target_price] / historical_max - 1.0
    historical_dd = daily_drawdown.cummin()
    MDD = historical_dd.min()

    # VOL = Volatility 휘발성, 변동성
    VOL = np.std(df[daily_rtn]) * np.sqrt(252.)

    # Sharp = ex-post Sharp ratio, 사후적 사프 비율
    # 샤프지수와 같은 공식을 사용하지만, 자산의 실현 수익률을 사용한다는 점에서 다르다.
    # 샤프지수는 위험 대비 수익성 지표를 의미함
    # 실현 수익률의 산술 평균 / 실현 수익률의 변동성으로 계산함
    # 보통 1 이상이면 좋다고 판단함

    Sharp = np.mean(df[daily_rtn]) / VOL

    return CAGR, Sharp, VOL, MDD
    


In [ ]:
data = yf.Ticker('MSFT')
df = data.history(period='1d', start=str_date_from, end=str_date_to)['Close'].reset_index().set_index('Date')
df.head()

In [ ]:
df.index[-1]

In [ ]:
select_stock_dict = dict()

CAGR_criteria = 0.05 # 연평균 복리 수익률(Compound Annual Growth Rate)
Sharp_criteria = 0.01  # 샤프 지수는 0.01 이상 되면 좋다고 판단한다. (VOL을 곱하기 100 해주면 Sharp는 1)
VOL_criteria = 0.60  # 변동성 지수는 60 이상 -> 하루하루 변동성이 크면서 성장해야 단타를 자주 하지 않을까?
MDD_criteria = -0.90  # 최대 낙폭은 1년 이내에 50%까지 하락한 경우가 있으면 허용한다.
target_price = 'Close'  # 일별 종가를 기준으로 계산한다.

for ticker in ticker_dict[market_list[0]]:
    df = yf.Ticker(ticker).history(period='1y')
    # df = fdr.DataReader(ticker, str_date_from, str_date_to)[target_price].reset_index().set_index('Date')
    
    if date_to in pd.to_datetime(df.index.tolist()):
        CAGR, Sharp, VOL, MDD = make_inverstment_performance_indicator(df, target_price, str_date_to)
        # print(ticker, CAGR, Sharp, VOL, MDD)
        if CAGR >= CAGR_criteria and Sharp >= Sharp_criteria and VOL >= VOL_criteria and MDD >= MDD_criteria:
            if ticker not in select_stock_dict:
                select_stock_dict[ticker] = dict(
                    CAGR = None,
                    Sharp = None,
                    VOL = None,
                    MDD = None
                )
            if ticker in select_stock_dict:
                select_stock_dict[ticker] = dict(
                    CAGR = CAGR,
                    Sharp = Sharp,
                    VOL = VOL,
                    MDD = -1*MDD*100
                )
select_stock_dict

In [ ]:
fdr.DataReader('AAPL', '2020-01-01', '2020-01-03')

In [ ]:
import pandas as pd

url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'

df_listing = pd.read_html(url, header=0)[0]

cols_ren = {'회사명':'Name', '종목코드':'Symbol', '업종':'Sector', '주요제품':'Industry', 

                    '상장일':'ListingDate', '결산월':'SettleMonth',  '대표자명':'Representative', 

                    '홈페이지':'HomePage', '지역':'Region', }

df_listing = df_listing.rename(columns = cols_ren)

df_listing['Symbol'] = df_listing['Symbol'].apply(lambda x: '{:06d}'.format(x))

df_listing['ListingDate'] = pd.to_datetime(df_listing['ListingDate'])

df_listing

In [ ]:
ms_df = fdr.DataReader(nasdaq_ticker_df.iloc[0]['Symbol'], str_start_date, str_today_date)
ms_df.head()

In [ ]:
price_df = ms_df['Close'].reset_index().set_index('Date')

In [ ]:
from_date = '2020-01-01'
to_date = '2021-01-01'
price_df.loc[from_date:to_date].plot(figsize=(16,9))

In [ ]:
# 일별 수익률
price_df['daily_rtn'] = price_df['Close'].pct_change()
price_df.head()

In [ ]:
# 바이앤 홀드 전략의 누적 곱 계산
price_df['st_rtn'] = (1 + price_df['daily_rtn']).cumprod()
price_df.head()

In [ ]:
price_df['st_rtn'].plot(figsize=(16,9))

In [ ]:
base_date = '2020-04-01'
tmp_df = price_df.loc[base_date:,['st_rtn']] / price_df.loc[base_date,['st_rtn']]
last_date = tmp_df.index[-1]
print('누적 수익 : ', tmp_df.loc[last_date, 'st_rtn'])
tmp_df.plot(figsize=(16,9))

In [ ]:
### 투자 성과 분석 지표
# 1. 연평균 복리 수익률
# 기하 평균 N 개의 변수를 모두 곱한 값의 제곱근
# 가장 최근일의 최종 누적 수익률의 누적 연도 제곱근을 구한다. 금융공학에서 1년은 252 영업일로 계산
# ** 제곱
CAGR = price_df.loc[str_today_date, 'st_rtn'] ** (252/len(price_df.index)) - 1
CAGR

In [ ]:
# 2. 최대 낙폭(MDD Maximum Daily Drawdown)
historical_max = price_df['Close'].cummax()
daily_drawdown = price_df['Close'] / historical_max - 1.0
historical_dd = daily_drawdown.cummin()
historical_dd.plot()


In [ ]:
historical_dd[-1]

In [ ]:
# 3. 변동성
# 변동성은 금융 자산의 방향성에 대한 불확실성과 가격 등락에 대한 위험 예상 지표로 해석
# 수익률의 표준편차를 변동성으로 계산함
VOL = np.std(price_df['daily_rtn']) * np.sqrt(252.)
VOL

In [ ]:
# 4. 샤프 지수
# 샤프 지수는 위험 대비 수익성 지표라고 볼 수 있다.
# Ra 는 자산 수익률 Rb 는 무위험 수익률
# 사후적 샤프 비율 ex-post Sharpe ratio
# 공식은 같지만 자산의 실현 수익률을 사용한다는 점에서 샤프 지수와 다름
# 실현 수익률의 산술 평균 / 실현 수익률의 변동성으로 계산함
Sharpe = np.mean(price_df['daily_rtn']) / np.std(price_df['daily_rtn']) * np.sqrt(252.)
Sharpe

In [ ]:
import threading, time, signal
from datetime import timedelta

wait_time_seconds = 1

class ProgramKilled(Exception):
    pass

def foo():
    print(time.ctime())

def signal_handler(signum, frame):
    raise ProgramKilled

class Job(threading.Thread):
    def __init__(self, interval, execute, *args, **kwargs):
        threading.Thread.__init__(self)
        self.daemon = False
        self.stopped = threading.Event()
        self.interval = interval
        self.execute = execute
        self.args = args
        self.kwargs = kwargs
    
    def stop(self):
        self.stopped.set()
        self.join()

    def run(self):
        while not self.stopeed.wait(self.interval.total_seconds()):
            self.execute(*self.args, **self.kwargs)


if __name__ == "__main__":
    signal.signal(signal.SIGTERM, signal_handler)
    signal.signal(signal.SIGINT, signal_handler)
    job = Job(interval=timedelta(seconds=wait_time_seconds), execute=foo)
    job.start()

    while True:
        try:
            time.sleep(1)
        except ProgramKilled:
            print("Program killed: running cleanup code")
            job.stop()
            break

In [ ]:
import requests
from public_data_config import apisdata

def post_message(token, channel, text):
    response = requests.post("https://slack.com/api/chat.postMessage",
        headers={"Authorization": "Bearer "+token},
        data={"channel": channel,"text": text}
    )
    print(response)
 
myToken = apisdata['slack']['token']
channel = apisdata['slack']['channel']
 
post_message(myToken,channel,"test_message")